In [1]:
%ls

Alice in Wonderland.pdf
Alice's Adventures in Wonderland by Lewis Carroll.zip
Fairy Tales from Hans Christian Andersen.pdf
Moby Dick; Or, The Whale by Herman Melville.zip
Pride and Prejudice by Jane Austen.zip
README.md
Romeo and Juliet by William Shakespeare.zip
a Little Princess by Frances Hodgson Burnett.zip
crime and punishment.zip
download_from_links.ipynb
game of thrones.csv
lesmisrables.zip
light-novel-titles.csv
lightnovellinks.zip
lotr characters.zip
mobydick.zip
some novels.zip
somelightnovels.zip
war and peace.zip


In [16]:
import requests
from urllib.request import urlopen
from urllib.parse import quote_plus
from bs4 import BeautifulSoup
import re
import pandas as pd
from tqdm import tqdm

In [32]:
switch = True
count = 88
while switch:
    url = f'https://www.gutenberg.org/cache/epub/{count}/pg{count}.txt'
    try:
        response = requests.get(url)
        if response.status_code == 200:
            text = response.text
            title_line = text.split('\n', 1)[0]
            pattern = '[^a-zA-Z0-9+]'
            title = title_line.replace('The Project Gutenberg eBook of ', '').strip()
            title = re.sub(pattern, '', title)

            # Save the text to a file
            with open(f'./gutenberg_crawled/{count}_{title}.txt', 'w', encoding='utf-8') as file:
                file.write(text)

            # if count == 1:
            #     print(f'First 10 characters of the book: {text[:10]}')
            count += 1
        else:
            print(f'Failed to fetch text for count {count}')
            switch = False

    except Exception as e:
        print(f'An error occurred: {e}')
        continue
    if count == 70000:
        switch = False

Failed to fetch text for count 114
